## Prepare Environment

In [ ]:
# !pip3 install pymongo
# !pip3 install mongoengine
# !pip3 install Faker
# !pip3 install mongomock

In [ ]:
# %load_ext nb_black
# %load_ext autoreload
# %autoreload 2

In [1]:
# Prepare environment for importing from src
import sys
import os

sys.path.insert(0, "..")

## Import Dependencies 

In [2]:
import random
import datetime

from mongoengine import connect, get_connection

from src.data import initialize_db
from src.utils import drop_db

## Connect to Mock DB

In [3]:
from pymongo import MongoClient

client = connect("assignment", host="mongodb://127.0.0.1:27017")

In [4]:
if not os.environ.get("TEST"):
    drop_db(client, "assignment")

## Generate Fake Data & Insert Them to DB

In [5]:
if not os.environ.get("TEST"):
    initialize_db()

In [6]:
print(client.assignment.list_collection_names())
print(client.assignment.patient.find_one())

['company', 'sale', 'user', 'driver', 'contract', 'pharmacy', 'product', 'doctor', 'drug', 'product_item', 'prescription', 'comment', 'patient']
{'_id': ObjectId('601bc689698b03d5b85a33ed'), 'national_id': '3389083863', 'first_name': 'Shane', 'last_name': 'Ramirez', 'address': '65423 Rogers Common Apt. 594\nBlairmouth, ME 72858', 'birthdate': datetime.datetime(1978, 3, 5, 0, 0), 'password': 'raymond57', 'doctor_id': ObjectId('601bc688698b03d5b85a33ec')}


## Examples

In [7]:
print(client.assignment.drug.find_one({"formula": "CH3COOH"}))

{'_id': ObjectId('601bc68e698b03d5b85a3780'), 'name': 'Midodrine Hydrochloride', 'company_id': ObjectId('601bc68b698b03d5b85a34b7'), 'formula': 'CH3COOH'}


In [8]:
list(
    client.assignment.drug.aggregate(
        [{"$group": {"_id": "$formula", "count": {"$sum": 1}}}]
    )
)

[{'_id': 'K2SO4', 'count': 2},
 {'_id': 'KCN', 'count': 4},
 {'_id': 'C6H6O', 'count': 7},
 {'_id': 'H2CO3', 'count': 8},
 {'_id': 'H3PO3', 'count': 5},
 {'_id': 'ZnSO4', 'count': 3},
 {'_id': 'Ar', 'count': 2},
 {'_id': 'C3H6O', 'count': 2},
 {'_id': 'C2H3N', 'count': 4},
 {'_id': 'CH2O', 'count': 7},
 {'_id': 'KHCO3', 'count': 8},
 {'_id': 'NH4I', 'count': 3},
 {'_id': 'CH3CHOHCH3', 'count': 2},
 {'_id': 'CH3CO2K', 'count': 4},
 {'_id': 'CaCO3', 'count': 1},
 {'_id': 'Na2HPO4', 'count': 4},
 {'_id': 'C4H10', 'count': 4},
 {'_id': 'HBr', 'count': 4},
 {'_id': 'FeSO4', 'count': 4},
 {'_id': 'NaOH', 'count': 5},
 {'_id': 'NH4Cl', 'count': 6},
 {'_id': 'CCl4', 'count': 5},
 {'_id': 'KSCN', 'count': 3},
 {'_id': 'C6H14', 'count': 2},
 {'_id': 'NH4Br', 'count': 1},
 {'_id': 'KBr', 'count': 5},
 {'_id': 'Na3PO4', 'count': 6},
 {'_id': 'C3H7NO2', 'count': 4},
 {'_id': 'H2O2', 'count': 3},
 {'_id': 'N2O', 'count': 1},
 {'_id': 'Na2SO3', 'count': 2},
 {'_id': 'HIO', 'count': 3},
 {'_id': 'C27H

In [9]:
client.assignment.patient.aggregate(
    [
        {
            "$lookup": {
                "from": "doctor",
                "localField": "doctor_id",
                "foreignField": "_id",
                "as": "doctor",
            }
        },
        {"$match": {"doctor.first_name": "Robert"}},
        {"$count": "patients"},
    ]
).next()

{'patients': 9}

## Query Assignments

In [19]:
# نام داروخانه هایی که شماره تلفن آنها با 1+ شروع می شود
print("##1##")
a1 = list(
    client.assignment.pharmacy.find(
        filter={'telephone':{'$regex':'\+1*'}},  # Complete the filter
        projection={"name": 1, "_id": 0, 'telephone': 1},
    )
)
print(a1)

##1##
[{'name': 'Rogers-Cardenas', 'telephone': '+1-328-708-3172x788'}, {'name': 'Jones, Fox and Romero', 'telephone': '+1-531-952-0585x277'}, {'name': 'Lee, Moore and Ferrell', 'telephone': '+1-506-098-3584x168'}, {'name': 'Ellison-Arias', 'telephone': '+1-721-903-4043x8427'}, {'name': 'Lopez-Buchanan', 'telephone': '+1-329-016-2047'}, {'name': 'Weber-Zamora', 'telephone': '+1-872-333-5104'}, {'name': 'Kennedy Inc', 'telephone': '+1-963-217-4357x483'}, {'name': 'Ramirez Group', 'telephone': '+1-851-939-3272x7281'}, {'name': 'Fry-Douglas', 'telephone': '+1-260-586-9427x674'}, {'name': 'Smith, Kemp and Hughes', 'telephone': '+1-456-339-1729x94036'}, {'name': 'Nicholson, Smith and Hernandez', 'telephone': '+1-471-855-9407'}]


In [31]:
# شماره ملی افرادی که بعد از تاریخ datetime.datetime(2000, 1, 1, 0, 0) متولد شده اند
print("##2##")
a2 = list(
    client.assignment.patient.find(
        filter={"birthdate" : {'$gt' : datetime.datetime(2000, 1, 1, 0, 0)}},  # Complete the filter
        projection={"national_id": 1, "_id": 0},
    )
)
print(a2)

##2##
[{'national_id': '7510799118'}, {'national_id': '8069901627'}, {'national_id': '7059640165'}, {'national_id': '9599534283'}, {'national_id': '7294667116'}, {'national_id': '5899441715'}, {'national_id': '9812387800'}, {'national_id': '4271739627'}, {'national_id': '2091241862'}, {'national_id': '0249724399'}, {'national_id': '4024240338'}, {'national_id': '9074260880'}, {'national_id': '3578243477'}, {'national_id': '0961701595'}, {'national_id': '8114578870'}, {'national_id': '8816502544'}, {'national_id': '1123274897'}, {'national_id': '4815269255'}, {'national_id': '8519537749'}, {'national_id': '5700412918'}, {'national_id': '2546862728'}, {'national_id': '8864078802'}, {'national_id': '0186074290'}, {'national_id': '2365401814'}]


In [ ]:
# تعداد نسخه هایی که دارای حداقل 15 دارو هستند
print("##3##")
a3 = client.assignment.prescription.find(
    filter={}  # Complete the filter
).count()
# print(a3)

In [ ]:
# کد ملی بیمارانی که اسم پزشک آنها "Robert" است
print("##4##")
a4 = list(
    client.assignment.patient.aggregate(
        [  # Complete the pipeline
        ]
    )
)
# print(a4)

In [ ]:
# نام داروخانه ای که دارویی به گرانترین قیمت به آن فروخته شده است
print("##5##")
a5 = client.assignment.sale.aggregate(
    [  # Complete the pipeline
    ]
).next()
# print(a5)

In [ ]:
# نام و فرمول پنج دارویی که گران ترین قیمت برای آنها ثبت شده است
print("##6##")
a6 = list(
    client.assignment.sale.aggregate(
        [  # Complete the pipeline
        ]
    )
)
# print(a6)

In [ ]:
# نام تمام داروهایی که در تاریخ datetime.datetime(2020, 9, 23, 0, 0) تجویز شده اند
print("##7##")
a7 = list(
    client.assignment.prescription.aggregate(
        [  # Complete the pipeline
        ]
    )
)
# print(a7)

In [ ]:
# نام تمام کارخانه هایی که داروی با فرمول "C2H6Na4O12" را تولید می کنند
print("##8##")
a8 = list(
    client.assignment.drug.aggregate(
        [  # Complete the pipeline
        ]
    )
)
# print(a8)

In [ ]:
# کاربرانی که در سبد آنها ده BasketItem وجود دارد
print("##9##")
a9 = list(
    client.assignment.user.find(
        filter={},  # Complete the filter
        projection={"email": 1, "_id": 0},
    )
)
# print(a9)

In [ ]:
print("##10##")
a10 = client.assignment.product_item.aggregate(
    [  # Complete the pipeline
    ]
).next()
# print(a10)

In [ ]:
# شماره ملی رانندگانی که پلاک آنها به 25 ختم می شود
print("##11##")
a11 = list(
    client.assignment.driver.find(
        filter={},  # Complete the filter
        projection={"_id": 0, "national_id": 1},
    )
)
# print(a11)

In [ ]:
print("##12##")
a12 = list(
    client.assignment.comment.find(
        filter={},  # Complete the filter
        projection={"_id": 0, "text": 1},
    )
)
# print(a12)

In [ ]:
print("##13##")
a13 = client.assignment.comment.aggregate(
    []  # Complete the pipeline
).next()
# print(a13)

In [ ]:
answers = {
    "a1": a1,
    "a2": a2,
    "a3": a3,
    "a4": a4,
    "a5": a5,
    "a6": a6,
    "a7": a7,
    "a8": a8,
    "a9": a9,
    "a10": a10,
    "a11": a11,
    "a12": a12,
    "a13": a13,
}

In [ ]:
import json
with open("answers.json", "r") as json_file:
    target = json.load(json_file)

In [ ]:
correct = 0
for i in range(1, 14):
    if answers["a{}".format(i)] == target["a{}".format(i)]:
        print("Query {:2d} Correct!".format(i))
        correct += 1
    else:
        print("Query {:2d} Wrong!".format(i))
print(correct)

## Print Result to File  

In [ ]:
# Set your student number
student_number = 90000000
file_path = os.path.join(
    os.getenv("OUTPUT_DIR", "."), "{}.json".format(student_number)
)
with open(file_path, "w") as file:
    corrects = []
    wrongs = []
    for i in range(1, 14):
        if answers["a{}".format(i)] == target["a{}".format(i)]:
            corrects.append(i)
        else:
            wrongs.append(i)
    json.dump({"corrects": corrects, "wrongs": wrongs, "score": len(corrects)}, file)